# Semantic Segmentation with Neural Networks

In this session we will see how to build a simple ConvNet for a toy semantic segmentation problem. We train the network from scratch and we will see the particularities of the problem. Finally, we will use a state of the art object detection network on some real-world images.

## A toy example

We will deal with images with simple objects of different shapes and colors. Our goal is: given an image with objects of different shapes and colors, classify each pixel with a label corresponding the type of object it belongs to.

### Data preparation

We will create the dataset ourselves with the following code. Our dataset will be composed of images containing objects of three different shapes (triangles, circles and squares). These will be of different sizes, colours, and they might overlap. Our task is to predict the object shape for each pixel of the image.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
palette = {(0,0,0):0, (0,0,255):1,(0,255,0):2,(255,0,0):3}
def convert_from_color_segmentation(arr_3d, palette, image_height=32, image_width=32):

    reshape_array = np.reshape(arr_3d, [image_height * image_width, 3])

    #still too slow!!
    arr_2d = np.fromiter([palette.get((x[0], x[1], x[2]), 0) for x in reshape_array],
                         reshape_array.dtype)

    return np.reshape(np.asarray(arr_2d), arr_3d.shape[0:2])

In [ ]:
import cairo
num_imgs = 5000

img_size = 32
min_object_size = 4
max_object_size = 16
num_objects = 4

shape_labels = ['rectangle', 'circle', 'triangle']
num_shapes = len(shape_labels)

imgs = np.zeros((num_imgs, img_size, img_size, 4), dtype=np.uint8)  # format: BGRA
masks = np.zeros((num_imgs, img_size, img_size, 4), dtype=np.uint8)
masks_decoded = []
shapes = np.zeros((num_imgs, num_objects), dtype=int)
colors = np.zeros((num_imgs, num_objects), dtype=int)

colors = [[0,0,1],[0,1,0],[1,0,0],[1,0,1],[1,1,0]]
num_colors = len(colors)

for i_img in range(num_imgs):
    surface = cairo.ImageSurface.create_for_data(imgs[i_img], cairo.FORMAT_ARGB32, img_size, img_size)
    surface_mask = cairo.ImageSurface.create_for_data(masks[i_img], cairo.FORMAT_ARGB32, img_size, img_size)
    
    cr = cairo.Context(surface)
    cr_mask = cairo.Context(surface_mask)
    # Fill background white.
    cr.set_source_rgb(1, 1, 1)
    cr.paint()
    
    cr_mask.set_source_rgb(0,0,0)
    cr_mask.paint()
    
    # Draw random shapes.
    for i_object in range(num_objects):
        shape = np.random.randint(num_shapes)
        shapes[i_img, i_object] = shape
        if shape == 0:  # rectangle
            w, h = np.random.randint(min_object_size, max_object_size, size=2)
            x = np.random.randint(0, img_size - w)
            y = np.random.randint(0, img_size - h)
            cr.rectangle(x, y, w, h)
            cr_mask.rectangle(x, y, w, h)
            cr_mask.set_source_rgb(0,0,1)
            cr_mask.fill()
        elif shape == 1:  # circle   
            r = 0.5 * np.random.randint(min_object_size, max_object_size)
            x = np.random.randint(r, img_size - r)
            y = np.random.randint(r, img_size - r)
            cr.arc(x, y, r, 0, 2*np.pi)
            cr_mask.arc(x, y, r, 0, 2*np.pi)
            cr_mask.set_source_rgb(0,1,0)
            cr_mask.fill()
        elif shape == 2:  # triangle
            w, h = np.random.randint(min_object_size, max_object_size, size=2)
            x = np.random.randint(0, img_size - w)
            y = np.random.randint(0, img_size - h)
            cr.move_to(x, y)
            cr.line_to(x+w, y)
            cr.line_to(x+w, y+h)
            cr.line_to(x, y)
            cr.close_path()
            
            cr_mask.move_to(x, y)
            cr_mask.line_to(x+w, y)
            cr_mask.line_to(x+w, y+h)
            cr_mask.line_to(x, y)
            cr_mask.close_path()
            
            cr_mask.set_source_rgb(1,0,0)
            cr_mask.fill()
        
        # TODO: Introduce some variation to the colors by adding a small random offset to the rgb values.
        color = np.random.randint(num_colors)
        r,g,b = colors[color]
        max_offset = 0.3
        r_offset, g_offset, b_offset = max_offset * 2. * (np.random.rand(3) - 0.5)
        cr.set_source_rgb(r-max_offset+r_offset, g+g_offset, b+b_offset)
        cr.fill()
    masks_decoded.append(convert_from_color_segmentation(masks[i_img][:,:,0:3],palette))
        
imgs = imgs[..., 2::-1]
masks_decoded = np.array(masks_decoded)

In [ ]:
masks_decoded.shape, imgs.shape

Let's look at one of the samples we created.

In [ ]:
plt.imshow(masks_decoded[2])
plt.show()
plt.imshow(imgs[2])

In [ ]:
from keras.utils import to_categorical
masks_decoded = masks_decoded.reshape(-1,1)
masks_decoded_cat = to_categorical(masks_decoded,num_classes = len(shape_labels) + 1)
masks_decoded_cat = masks_decoded_cat.reshape(num_imgs,img_size,img_size,len(shape_labels) + 1)

In [ ]:
X = (imgs - np.mean(imgs)) / np.std(imgs)
X.shape, np.mean(X), np.std(X)
X.shape

In [ ]:
i_train = int(0.6 * num_imgs)
i_val = int(0.7 * num_imgs)

train_X = X[:i_train]
val_X = X[i_train:i_val]
test_X = X[i_val:]
train_y = masks_decoded_cat[:i_train]
val_y = masks_decoded_cat[i_train:i_val]
test_y = masks_decoded_cat[i_val:]
test_imgs = imgs[i_val:]

### Model

We will build a simple model composed of 4 convolutional layers.

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D

In [ ]:
model = Sequential([
        Conv2D(filters=64,kernel_size=3, input_shape=(X.shape[1:]),activation='relu',padding='same'), 
        Conv2D(filters=64,kernel_size=3,activation='relu', padding='same'),
        Conv2D(filters=32,kernel_size=3,activation='relu', padding='same'),
        Conv2D(filters=num_shapes+1,kernel_size=3,activation='softmax',padding='same')
    ])
model.compile('adadelta', 'categorical_crossentropy')
model.summary()

### Training

Let's train !

In [ ]:
n_epochs = 50
history = model.fit(train_X,train_y,batch_size=512,epochs=n_epochs,validation_data=(val_X,val_y))

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

### Testing

In [ ]:
preds = model.predict(test_X)

In [ ]:
# Triangle:1, Circle:2, Square:3

for i,pred in enumerate(preds[0:5]):
    fig, (ax0, ax1,ax2) = plt.subplots(ncols=3,figsize=(30,10))
    argmax_pred = np.argmax(pred,axis=-1)
    cf0 = ax0.imshow(test_imgs[i])
    fig.colorbar(cf0,ax=ax0)
    cf1 = ax1.imshow(argmax_pred,vmin=0,vmax=3,cmap='magma')
    fig.colorbar(cf1,ax=ax1)
    cf2 = ax2.imshow(np.argmax(test_y[i],axis=-1),vmin=0,vmax=3,cmap='magma')
    fig.colorbar(cf2,ax=ax2)
    plt.show()

## Semantic Segmentation of Real-World Images

Let's dive into a network that can deal with real images !

We will use a pretrained semantic segmentation network to obtain results on some of our test images. The network we chose is the one introduced in:

Jonathan Long, Evan Shelhamer, Trevor Darrell. [Fully Convolutional Networks for Semantic Segmentation](www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf
). CVPR 2015


FCN is composed of a ConvNet from which final classifier layer has been removed and all fully connected layers have been converted to convolutions. We append a 1x1 convolution with channel dimension 21 to predict scores for each of the PASCAL classes (including background) at each of the coarse output locations, followed by a deconvolution layer to bilinearly upsample the coarse outputs to pixel-dense outputs.

FCN combines layers of the feature hierarchy and refines the spatial precision of the output. While fully convolutionalized classifiers can be fine-tuned to segmentation, and even score highly on the standard metric, their output is dissatisfyingly coarse. The 32 pixel stride at the final prediction layer limits the scale of detail in the upsampled output.
This issue is addressed by adding skips that combine the final prediction layer with lower layers with finer strides. As they see fewer pixels, the finer scale predictions should need fewer layers, so it makes sense to make them from shallower net outputs. Combining fine layers and coarse layers lets the model make local predictions that respect global structure.
We first divide the output stride in half by predicting from a 16 pixel stride layer. We add a 1x1 convolution layer on top of pool4 to produce additional class predictions. We fuse this output with the predictions computed on top of conv7 (convolutionalized fc7) at stride 32 by adding a 2x upsampling layer and summing both predictions. Finally, the stride 16 predictions are upsampled back to the image.

We call this net FCN-16s.

Remark :
The original paper mention that FCN-8s (slightly more complex architecture) does not provide much improvement so we stopped at FCN-16s

Note: the code is adapted from [this notebook](https://github.com/mzaradzki/neuralnets/blob/master/vgg_segmentation_keras/fcn16s_segmentation_keras2.ipynb).

In [ ]:
import sys
sys.path.insert(0,'util/fcn/')
from util.fcn.fcn import fcn32_blank, fcn_32s_to_16s, prediction

In [ ]:
image_size = 64*8 # INFO: initially tested with 256, 448, 512

In [ ]:
fcn32model = fcn32_blank(image_size)
fcn16model = fcn_32s_to_16s(fcn32model)
fcn16model.summary() # visual inspection of model architecture


Let's now load the pretrained weights to the model we created. The model was trained with matconvnet, so we need to load the `.mat` file and decode the weights so that our keras model can understand them

In [ ]:
from scipy.io import loadmat
data = loadmat('data/models/pascal-fcn16s-dag.mat', matlab_compatible=False, struct_as_record=False)
l = data['layers']
p = data['params']
description = data['meta'][0,0].classes[0,0].description

We can see that the `.mat` file stores information of the layers, the parameters (pooling layers not included here) and the output categories of the network.

In [ ]:
l.shape, p.shape, description.shape

In [ ]:
class2index = {}
for i, clname in enumerate(description[0,:]):
    class2index[str(clname[0])] = i
    
print(sorted(class2index.keys()))

In [ ]:
for i in range(0, p.shape[1]-1, 2):
    print(i,
          str(p[0,i].name[0]), p[0,i].value.shape,
          str(p[0,i+1].name[0]), p[0,i+1].value.shape)

Here is the function that we will use to copy the weights from the `.mat` file to the keras model. We basically will check that the layer names & layer shapes match, and will set the weights and biases with `model.layers[idx].set_weights`

In [ ]:
def copy_mat_to_keras(kmodel):
    
    kerasnames = [lr.name for lr in kmodel.layers]

    prmt = (0, 1, 2, 3) # WARNING : important setting as 2 of the 4 axis have same size dimension
    
    for i in range(0, p.shape[1]-1, 2):
        matname = '_'.join(p[0,i].name[0].split('_')[0:-1])
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            print('found : ', (str(matname), kindex))
            l_weights = p[0,i].value
            l_bias = p[0,i+1].value
            f_l_weights = l_weights.transpose(prmt)
            if False: # WARNING : this depends on "image_data_format":"channels_last" in keras.json file
                f_l_weights = np.flip(f_l_weights, 0)
                f_l_weights = np.flip(f_l_weights, 1)
            print(f_l_weights.shape, kmodel.layers[kindex].get_weights()[0].shape)
            assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
            assert (l_bias.shape[1] == 1)
            assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
            assert (len(kmodel.layers[kindex].get_weights()) == 2)
            kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])
        else:
            print('not found : ', str(matname))

In [ ]:
copy_mat_to_keras(fcn16model)

We are now ready to forward an image through the network and get some predictions

In [ ]:
from PIL import Image

im = Image.open('data/images/bus_creative_commons.jpg')
im = im.resize((image_size,image_size))

In [ ]:
plt.imshow(np.asarray(im))
print(np.asarray(im).shape)

In [ ]:
preds = prediction(fcn16model, im, transform=False)

We will display the class with maximum probability for each pixel in the output.

In [ ]:
print(preds.shape)
imclass = np.argmax(preds, axis=3)[0,:,:]
print(imclass.shape)
plt.figure(figsize = (15, 7))
plt.subplot(1,3,1)
plt.imshow( np.asarray(im) )
plt.subplot(1,3,2)
plt.imshow( imclass )
plt.subplot(1,3,3)
plt.imshow( np.asarray(im) )
masked_imclass = np.ma.masked_where(imclass == 0, imclass)
#plt.imshow( imclass, alpha=0.5 )
plt.imshow( masked_imclass, alpha=0.5 )

Let's see the list of classes that were found in the image (those that obtained the maximum probability in at least one of the pixels)

In [ ]:
for c in np.unique(imclass):
    print(c, str(description[0,c][0]))

Now we will pick some of the classes and display its activation values for all pixels in the image. Here we are not displaying the class with maximum value for each pixel, but the actual value for all pixels given a class.

In [ ]:
from scipy.misc import bytescale
bspreds = bytescale(preds, low=0, high=255)
print (bspreds.shape)
plt.figure(figsize = (15, 7))
plt.subplot(2,3,1)
plt.imshow(np.asarray(im))
plt.subplot(2,3,3+1)
plt.imshow(bspreds[0,:,:,class2index['background']], cmap='seismic')
plt.subplot(2,3,3+2)
plt.imshow(bspreds[0,:,:,class2index['bus']], cmap='seismic')
plt.subplot(2,3,3+3)
plt.imshow(bspreds[0,:,:,class2index['person']], cmap='seismic')

We can see the segmentation of the person in the image being much more accurate here. However, the probability of the category `bus` appeared to be higher in most pixels.